In [1]:
import pandas as pd
import numpy as np

In [2]:
from utils import plotting, transform, ingest
from forecast import stats, univariate

In [3]:
from dateutil.relativedelta import relativedelta

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={"figure.figsize": (11, 8.3)})

In [5]:
cols = ["HOUST", "PERMIT"]
regions = ["NE", "MW", "S", "W"]

regional_hst = [f"HOUST{r}" for r in regions]

In [6]:
raw_df = ingest.import_fred()

parsed_df = transform.standard(raw_df)

Using cached data at data/fred_data.csv


In [41]:
lags = 36

national_houst = parsed_df["HOUST"]

train = national_houst[:"1999-12-01"]
test = national_houst["2000-01-01":]

In [ ]:
predictor = univariate.rf_forecast(train, lags = lags, n_jobs = 3)

In [ ]:
last_day = train.index[-1]
append_period = last_day - relativedelta(months=lags - 1)
first_lags = train[append_period:last_day].to_numpy()

In [ ]:
def iterative_forecast(pred, init, periods = 1):
    
    X = init.copy()
    y_hat = np.zeros((periods, 3))
    
    for i in range(periods):
        forecast = pred(X)
        
        X[:-1] = X[1:]
        X[-1] = forecast[1]
        
        y_hat[i] = forecast
            
    return y_hat

In [ ]:
y_hat = iterative_forecast(predictor, first_lags, lags)

In [ ]:
df_forecast = pd.DataFrame(y_hat, columns = ["lb", "y", "ub"], index=test.index[:len(y_hat)])

In [ ]:
fig, ax = plt.subplots()

sns.lineplot(data=train.loc["2007-01-01":], ax = ax, color ="r", marker = "o")
sns.lineplot(data=test.iloc[:len(df_forecast)], ax = ax, color="r", marker = "o")

plt.axvline(test.index[0], linestyle="--")

sns.lineplot(data=df_forecast["y"], ax = ax, color="g", linestyle="--", marker = "o")
sns.lineplot(data=df_forecast["lb"], ax = ax, color="b", alpha = 0.5)
sns.lineplot(data=df_forecast["ub"], ax = ax, color="b", alpha = 0.5)

ax.fill_between(df_forecast.index, df_forecast["lb"], df_forecast["ub"], df_forecast["lb"], alpha=0.2)

fig.show()

In [ ]:
def plot_iter_forecast(predictor, test_data, full_data, init, periods = 1):
    y_hat = iterative_forecast(predictor, init, periods)
    